In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import folium

In [ ]:
df=pd.read_csv("busan_oldage_ratio.csv",index_col="행정구역",encoding="cp949")

# '비율'이라는 단어가 들어간 컬럼만 선택
selected_columns = df.columns[df.columns.str.contains('비율')]

# 선택된 컬럼으로 새로운 데이터프레임 생성
df1 = df[selected_columns]

df1

In [ ]:
# 각 연도별 비율 차이 계산
df['증가 속도'] = df1.iloc[:, -1] - df1.iloc[:, 0]

# 기간 설정
years = 2023 - 2008

# 평균 연간 증가 속도 계산
df['평균 연간 증가 속도'] = df['증가 속도'] / years

In [ ]:
df.head()

In [ ]:
df['adm_cd2'] = df.index.str.split('(').str[1].str.split(')').str[0]
df.head()

## 행정동 표시하기

In [ ]:
# GeoJSON 파일 경로
geojson_file = 'HangJeongDong_ver20230701.geojson'

# GeoPandas를 사용하여 GeoJSON 파일 읽기
gdf = gpd.read_file(geojson_file)

# "sido" 열의 값이 '26'인 행만 선택
selected_gdf = gdf[gdf["sido"] == '26']

# 선택된 GeoPandas 데이터프레임을 Folium 맵으로 변환
m = folium.Map(location=[selected_gdf['geometry'].centroid.y.mean(), selected_gdf['geometry'].centroid.x.mean()],
               zoom_start=10,
               tiles="cartodbpositron")

# 선택된 GeoPandas 데이터프레임의 각 지오메트리를 Folium 지오메트리로 추가
for _, geojson_data in selected_gdf.iterrows():
    folium.GeoJson(geojson_data['geometry']).add_to(m)

# Folium 맵을 표시
m

In [ ]:
selected_gdf

In [ ]:
# 데이터프레임과 geojson 데이터 결합
merged = selected_gdf.set_index('adm_cd2').join(df.set_index('adm_cd2'))
merged

In [ ]:
# 선택된 GeoPandas 데이터프레임을 Folium 맵으로 변환
m = folium.Map(location=[selected_gdf['geometry'].centroid.y.mean(), selected_gdf['geometry'].centroid.x.mean()],
               zoom_start=10,
               tiles="cartodbpositron")


folium.Choropleth(
    geo_data=merged.__geo_interface__,
    name='choropleth',
    data=merged.reset_index(),
    columns=['adm_nm', '증가 속도'],
    key_on='feature.properties.adm_nm',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='비율 증가 속도'
).add_to(m)

# 지도 출력
m


In [ ]:
# 선택된 GeoPandas 데이터프레임을 Folium 맵으로 변환
m = folium.Map(location=[selected_gdf['geometry'].centroid.y.mean(), selected_gdf['geometry'].centroid.x.mean()],
               zoom_start=10,
               tiles="cartodbpositron")


folium.Choropleth(
    geo_data=merged.__geo_interface__,
    name='choropleth',
    data=merged.reset_index(),
    columns=['adm_nm', '2023년10월_비율'], 
    key_on='feature.properties.adm_nm',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='2023년 10월 기준 고령 인구 비율'
).add_to(m)

# 지도 출력
m


## 두 컬럼의 값을 전체 지역의 75% 분위수 이상인 동을 선정
#### (비율은 조정 가능)

In [ ]:
# 75% 분위수 이상인 값을 가진 동을 찾음
serious_area = merged[(merged['증가 속도'] >= merged['증가 속도'].quantile(0.75)) & 
                      (merged['2023년10월_비율'] >= merged['2023년10월_비율'].quantile(0.75))]

# 심각한 지역의 adm_nm 출력
print(serious_area['adm_nm'])

In [ ]:
serious_area.head()

In [ ]:
# 선택된 GeoPandas 데이터프레임을 Folium 맵으로 변환
m = folium.Map(location=[serious_area['geometry'].centroid.y.mean(), serious_area['geometry'].centroid.x.mean()],
               zoom_start=10,
               tiles="cartodbpositron")

folium.Choropleth(
    geo_data=serious_area.__geo_interface__,
    name='choropleth',
    data=serious_area.reset_index(),
    columns=['adm_nm', '2023년10월_비율'],
    key_on='feature.properties.adm_nm',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='2023년 10월 고령 인구 비율'
).add_to(m)

# 지도 출력
m


In [ ]:
# '인구_기반_위험지역_선정_여부_75' 컬럼 추가
merged['인구_기반_위험지역_선정_여부_75'] = False

# serious_area에 해당하는 지역을 True로 표시
merged.loc[merged['adm_nm'].isin(serious_area['adm_nm']), '인구_기반_위험지역_선정_여부_75'] = True

merged

In [ ]:
num_true = merged[merged['인구_기반_위험지역_선정_여부_75'] == True].shape[0]
print("인구 기반 위험지역 선정 개수:", num_true)

In [ ]:
merged.to_csv("old_age_vel_serious_area.csv",encoding="cp949")

In [ ]:
# '2008년11월_비율'부터 '2023년10월_비율'까지의 컬럼 선택
selected_columns = [col for col in merged.columns if '2008년11월_비율' <= col <= '2023년10월_비율']

# 각 연월별 비율의 차이 (증가속도) 계산
adc = merged[selected_columns].diff(axis=1)

# 증가속도의 차이 (증가속도의 증감) 계산 후 평균 취하기
merged['증가속도의_증감'] = adc.diff(axis=1).mean(axis=1)

merged

In [ ]:
merged = merged.drop(['증감정도', '증가속도1'], axis=1)

In [ ]:
# 지도의 중심을 지정하기 위해 위도와 경도의 평균값을 계산
average_lat = merged['geometry'].centroid.y.mean()
average_lon = merged['geometry'].centroid.x.mean()

# Folium 지도를 생성
m = folium.Map([average_lat, average_lon], zoom_start=10,tiles="cartodbpositron")

# Choropleth 레이어를 추가
folium.Choropleth(
    geo_data=merged.set_index('adm_nm')['geometry'].__geo_interface__,
    name='choropleth',
    data=merged,
    columns=['adm_nm', '증가속도의_증감'],
    key_on='feature.id',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='증가속도의_증감'
).add_to(m)

m

In [ ]:
# 임계값 설정
증가속도의_증감_임계값 = merged['증가속도의_증감'].quantile(0.50)
증가속도_임계값 = merged['증가 속도'].quantile(0.50)
비율_임계값 = merged['2023년10월_비율'].quantile(0.50)

# 각 행별로 임계값을 초과하는지 확인하여 '속도_기반_위험지역_선정_여부_75' 컬럼 생성
merged['속도_기반_위험지역_선정_여부_75'] = (merged['증가속도의_증감'] > 증가속도의_증감_임계값) & (merged['증가 속도'] > 증가속도_임계값) & (merged['2023년10월_비율'] > 비율_임계값)

merged

In [ ]:
# '속도_기반_위험지역_선정_여부_75' 컬럼이 True인 행의 개수를 계산
위험지역_개수 = merged['속도_기반_위험지역_선정_여부_75'].sum()

print(f"고령화가 심각한 지역의 개수: {위험지역_개수}")


In [ ]:
import folium

# '속도_기반_위험지역_선정_여부_75'가 True인 행만 추출
위험지역_df = merged.loc[merged['속도_기반_위험지역_선정_여부_75'] == True]

# 지도의 중심을 지정하기 위해 위도와 경도의 평균값을 계산
average_lat = 위험지역_df['geometry'].centroid.y.mean()
average_lon = 위험지역_df['geometry'].centroid.x.mean()

# Folium 지도를 생성
m = folium.Map([average_lat, average_lon], zoom_start=10,tiles="cartodbpositron")

# Choropleth 레이어를 추가
folium.Choropleth(
    geo_data=위험지역_df.set_index('adm_nm')['geometry'].__geo_interface__,
    name='choropleth',
    data=위험지역_df,
    columns=['adm_nm', '증가속도의_증감'],
    key_on='feature.id',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='증가속도의_증감'
).add_to(m)

m


In [ ]:
merged.to_csv("old_age_vel_serious_area_final.csv",encoding="cp949")